In [3]:
import requests

url = "https://stujo-tic-tac-toe-stujo-v1.p.rapidapi.com/%7Bstate%7D/%7Bplayer%7D"

headers = {
	"X-RapidAPI-Key": "4ec23d0093mshae516a3b5277798p1ee8ffjsn075ee1e402ee",
	"X-RapidAPI-Host": "stujo-tic-tac-toe-stujo-v1.p.rapidapi.com"
}

response = requests.get(url, headers=headers)

print(response)

<Response [404]>


In [ ]:
import gym
from gym.utils import play 
from gym import wrappers
from gym.wrappers import GrayScaleObservation, RecordEpisodeStatistics, TimeLimit, ResizeObservation, FrameStack

import matplotlib.pyplot as plt
import time
from hashlib import md5
import os

def apply_wrappers(env):
    env = GrayScaleObservation(env)
    env = ResizeObservation(env, (84, 84))
    env = FrameStack(env, num_stack=4)
    env = TimeLimit(env, 500)
    env = RecordEpisodeStatistics(env)
    return env

In [ ]:
env = gym.make("ALE/Breakout-v5",  obs_type='rgb', frameskip=4)
env = apply_wrappers(env)

# env.metadata['render_fps'] = 30
episode_n = 500

frame_size = []
frame_size.extend(list(env.observation_space.shape))

agent = DQNAgent(frame_size, env.action_space.n, episode_n=episode_n)
#agent.load_model('/home/artem/atari_games/models/DQN_c11323c6ff7080195e86ee7b2e01d04b.pth')
# state, info = env.reset()
DQN_learning(agent, episode_n, visualize=False)
agent.save_model('/home/artem/atari_games/models/atari_model_1.pth')

In [4]:
import gymnasium
from gym.utils import play 
from gym import wrappers
from gym.wrappers import GrayScaleObservation, RecordEpisodeStatistics, TimeLimit, ResizeObservation, FrameStack

import matplotlib.pyplot as plt
import time
from hashlib import md5
import os
import gym_examples


os.chdir('/home/artem/atari_games')
os.getcwd()

env = gymnasium.make('gym_examples/GridWorld-v0', render_mode='human')

In [5]:
env.reset()
env.render()

In [ ]:
import numpy as np
import pygame

import gymnasium as gym
from gymnasium import spaces


class TicTacEnv(gym.Env):
    metadata = {"render_modes": ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None, size=3):
        self.size = size  # The size of the square grid
        self.window_size = size  # The size of the PyGame window
        self.step = 1

        # Observations are dictionaries with the agent's and the target's location.
        # Each location is encoded as an element of {0, ..., `size`}^2, i.e. MultiDiscrete([size, size]).
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                "target": spaces.Box(0, size - 1, shape=(2,), dtype=int),
                
            }
        )

        self.observation_space = spaces.Box(0, 2, shape=(self.size ** 2,), dtype=int)
        

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(9)

        """
        The following dictionary maps abstract actions from `self.action_space` to
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

    def _get_obs(self):
        return {"agent": self._agent_location, "target": self._target_location}

    def _get_info(self):
        return {
            "distance": np.linalg.norm(
                self._agent_location - self._target_location, ord=1
            )
        }

    def is_legal_action(self, action):
        return self.observation_space[action] == 0

    def _get_action_vector(self, action):
        num = 1 if step % 2 == 1 else 2
        return [num if i == action else 0 for i in range(self.size ** 2)]

    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's location uniformly at random
        self._agent_location = self.np_random.integers(0, self.size, size=2, dtype=int)

        # We will sample the target's location randomly until it does not coincide with the agent's location
        self._target_location = self._agent_location
        while np.array_equal(self._target_location, self._agent_location):
            self._target_location = self.np_random.integers(
                0, self.size, size=2, dtype=int
            )

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info

    def step(self, action):
        # Map the action (element of {0,1,2,3}) to the direction we walk in
        direction = self._action_to_direction[action]
        # We use `np.clip` to make sure we don't leave the grid
        self._agent_location = np.clip(
            self._agent_location + direction, 0, self.size - 1
        )
        # An episode is done iff the agent has reached the target
        terminated = np.array_equal(self._agent_location, self._target_location)
        reward = 1 if terminated else 0  # Binary sparse rewards
        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def render(self):
        if self.render_mode == "rgb_array":
            return self._render_frame()

    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode(
                (self.window_size, self.window_size)
            )
        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        canvas = pygame.Surface((self.window_size, self.window_size))
        canvas.fill((255, 255, 255))
        pix_square_size = (
            self.window_size / self.size
        )  # The size of a single grid square in pixels

        # First we draw the target
        pygame.draw.rect(
            canvas,
            (255, 0, 0),
            pygame.Rect(
                pix_square_size * self._target_location,
                (pix_square_size, pix_square_size),
            ),
        )
        # Now we draw the agent
        pygame.draw.circle(
            canvas,
            (0, 0, 255),
            (self._agent_location + 0.5) * pix_square_size,
            pix_square_size / 3,
        )

        # Finally, add some gridlines
        for x in range(self.size + 1):
            pygame.draw.line(
                canvas,
                0,
                (0, pix_square_size * x),
                (self.window_size, pix_square_size * x),
                width=3,
            )
            pygame.draw.line(
                canvas,
                0,
                (pix_square_size * x, 0),
                (pix_square_size * x, self.window_size),
                width=3,
            )

        if self.render_mode == "human":
            # The following line copies our drawings from `canvas` to the visible window
            self.window.blit(canvas, canvas.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # We need to ensure that human-rendering occurs at the predefined framerate.
            # The following line will automatically add a delay to keep the framerate stable.
            self.clock.tick(self.metadata["render_fps"])
        else:  # rgb_array
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(canvas)), axes=(1, 0, 2)
            )

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()

In [3]:
step = 1
d = {
    'a': 1 if step == 1 else 0
}
print(d['a'])
step = 0
print(d['a'])

1
1


In [5]:
[1 if i == 2 else 0 for i in range(2 ** 2)]

[0, 0, 1, 0]